In [1]:
# 用于获取最新的DTBS的A区中的 cz 纯债价值 pd 平底溢价率
# 由于原先的cz是每周获取一次，因此若需要每日更新一次则需要这个文件，考虑到wind数据容量有限选择增量更新
# cz这个数据感觉更新的相当晚，因此不要太早跑，安全起见5点之后甚至第二天跑

import pandas as pd
import numpy as np
import pickle
import time
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
with open(DTBS_path, 'rb') as f:
    DTBS = pickle.load(f)

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

In [2]:
# 找出cz为空的那些行
df = A_DF[A_DF['cz'].isnull()]

day_list = sorted(list(set(df['date'])))

need_query_mapping = {}

for day in day_list:
    need_query_mapping[day] = list(df[df['date']==day]['bond_code'])

need_query_mapping

{'2023-09-05': ['110088.SH',
  '113545.SH',
  '113631.SH',
  '123128.SZ',
  '123156.SZ',
  '127027.SZ',
  '127061.SZ',
  '110047.SH',
  '110063.SH',
  '110070.SH',
  '110087.SH',
  '110089.SH',
  '110092.SH',
  '111001.SH',
  '111002.SH',
  '111003.SH',
  '111007.SH',
  '111012.SH',
  '113025.SH',
  '113027.SH',
  '113046.SH',
  '113047.SH',
  '113058.SH',
  '113059.SH',
  '113064.SH',
  '113534.SH',
  '113535.SH',
  '113566.SH',
  '113591.SH',
  '113600.SH',
  '113615.SH',
  '113621.SH',
  '113623.SH',
  '113632.SH',
  '113636.SH',
  '113638.SH',
  '113640.SH',
  '113641.SH',
  '113649.SH',
  '113650.SH',
  '113653.SH',
  '113657.SH',
  '113663.SH',
  '118000.SH',
  '118005.SH',
  '118006.SH',
  '118008.SH',
  '118010.SH',
  '118018.SH',
  '118020.SH',
  '118022.SH',
  '118023.SH',
  '118028.SH',
  '118029.SH',
  '123012.SZ',
  '123013.SZ',
  '123031.SZ',
  '123046.SZ',
  '123052.SZ',
  '123076.SZ',
  '123078.SZ',
  '123085.SZ',
  '123091.SZ',
  '123106.SZ',
  '123118.SZ',
  '123124.S

In [3]:
from WindPy import w
w.start()

for day in day_list:
    
    bond_code_list = need_query_mapping[day]
    
    # 纯债价值列表
    bond_value_list = w.wsd(bond_code_list, "strbvalue", day, day, "").Data[0]
    
    print(bond_value_list)
    
    for bond_code_index in range(len(bond_code_list)):
        bond_code = bond_code_list[bond_code_index]
        bond_value = bond_value_list[bond_code_index]
        DTBS['A'][bond_code][day]['cz'] = bond_value
        DTBS['A'][bond_code][day]['pd'] = (DTBS['A'][bond_code][day]['csv'] / DTBS['A'][bond_code][day]['cz'] - 1) * 100

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
[96.024, 106.6343, 100.5712, 99.4802, 82.1376, 103.9779, 94.34, 111.5119, 107.5636, 106.8705, 96.4949, 97.9677, 94.8533, 94.4735, 100.7366, 93.0724, 89.3339, 80.049, 107.2317, 95.1054, 102.7562, 101.8715, 99.3752, 98.8157, 97.0605, 103.3309, 106.5092, 101.8089, 103.0433, 99.6798, 104.6107, 102.3798, 103.2731, 101.0518, 94.2406, 100.5322, 92.9602, 97.6457, 90.8253, 90.6705, 90.7199, 85.655, 88.5891, 98.9855, 89.2144, 85.4691, 90.7558, 78.8323, 77.4327, 86.6903, 95.8217, 89.1471, 88.4256, 75.5091, 101.6436, 103.4665, 97.7826, 101.3518, 101.659, 98.6154, 103.1796, 93.8529, 103.7135, 102.1208, 90.0418, 81.754, 95.4983, 85.2932, 91.9785, 84.8383, 95.9655, 89.1017, 81.2046, 89.011, 85.76, 107.8153, 116.6538, 103.7002, 103.648, 99.0676, 100.4561, 100.7614, 103.406

In [4]:
DTBS['A']['113545.SH']['2023-08-29']

{'cpr': 40.8,
 'dp': 118.749,
 'dl': 159.54899999999998,
 'ytm': -2.23,
 'bl': 9.865,
 'trt': 2.49,
 'yl': 2.129,
 'csp': 9.96,
 'ia': 1,
 'xx': 0,
 'hs': 0,
 'csv': 84.33734939759036,
 'qs': 0,
 'qs15': 0,
 'qs30': 0,
 'xx15': 3,
 'xx30': 3,
 'cz': 106.7029,
 'pd': -20.960583641503316}

In [5]:
# 保存上面的这些信息的更改
f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()